In [1]:
from data_tools.api import *
from utilscht.Data import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from volat_calcu import *
import pymysql
from joblib import Parallel,delayed
%config InlineBackend.figure_format ='retina'
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from reader import *


DB_INFO = dict(host='192.168.1.234',
               user='winduser',
               password='1qaz@WSX',
               db='wind')

conn = pymysql.connect(**DB_INFO, charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)

In [19]:
def apply_parallel(df_grouped, func, n_jobs=16, backend='loky', as_index=True, ratio=0.1):
    """
    This is for parallel between grouped generated by pd.DataFrame.groupby
    :param df_grouped:
    :param func:
    :param n_jobs:
    :param backend:
    :param kwargs:
    :return:
    """

    names = []
    groups = []
    for name, group in df_grouped:
        names.append(name)
        groups.append(group)

    results = Parallel(n_jobs=n_jobs, verbose=5, backend=backend, batch_size='auto') \
        (delayed(func)(group) for group in groups)

    return pd.concat(results, keys=names if as_index else None)

In [3]:
from data_tools.api import trade_days

trade_dates_all = trade_days.copy()
def get_prev_n_trade_date(trade_date, n):
    trade_date=str(trade_date)[0:10].replace('-','')
    pos = np.searchsorted(trade_dates_all, trade_date)
    try:
        assert pos >= n
        return str(trade_dates_all[pos - n])
    except AssertionError:
        return "20200101"

def get_next_n_trade_date(trade_date, n=1):
    trade_date=str(trade_date)[0:10].replace('-','')
    pos = np.searchsorted(trade_dates_all, trade_date, side='right')
    try:
        assert pos + n - 1 < len(trade_dates_all)
        return str(trade_dates_all[pos + n - 1])
    except AssertionError:
        return "20201231"

In [4]:
# 只取20交易日内触发止损止盈的，且20交易日内不重复买入同一支股票
def get_date_stop_pl(df,ratio = 0.1):
    sid = df["sid"].iloc[0]
    date = df["date"].iloc[0]
    hq_data = get_stk_bar(sid,start=date,end=get_next_n_trade_date(date,21),fields=["adj_close",'adj_high',"adj_low"])
    thr_up= hq_data["adj_close"].values[0]*(1+ratio)
    thr_down= hq_data["adj_close"].values[0]*(1-ratio)
    
    df=pd.DataFrame(index=[0],columns=["date_{}p_ud".format(int(ratio*100)),"up_down"])
    for dt in hq_data.index[1:21]:
        if hq_data.loc[dt,"adj_high"] > thr_up:
            df.iloc[0]=(str(dt)[0:10], 1)
            return df
        if hq_data.loc[dt,"adj_low"] < thr_down:
            df.iloc[0]=(str(dt)[0:10],-1)
            return df
    
    change_rate = hq_data.loc[dt,"adj_close"]/hq_data["adj_close"].values[0]-1
    df.iloc[0]=(str(dt)[0:10],change_rate)
    return df

def date_filter(df):
    use_dates=[]
    date_list = df["date"].values
    while(1):
        if len(date_list)==0:
            break
        else:
            date =  min(date_list)
            use_dates.append(date)
            dates_rm = get_trade_dates(date,min(get_next_n_trade_date(date,19),"20191231"))
            date_list=list(set(date_list)-set(dates_rm))
    return df[df["date"].isin(use_dates)]
    

## 统计白马股票日内穿网格的次数满足要求的日期

In [6]:

trade_dates=get_trade_dates('20180101',"20200331")
trade_dates=[str(i) for i in trade_dates]

df_cross_grid_count=pd.DataFrame()
for date in trade_dates:
    stock_pool=pd.read_csv(r"/share/xfzhang/to_colleague/to_yzhao/task2/{}/{}/task2_{}.csv".format(date[0:4],date[4:6],date[0:8]))["sid"]
    cross_grid=pd.read_excel(r"/home/ywang/proj_cross_grid/result/cross_count_b240/crosscount_summary_{}.xlsx".format(date))
    cross_grid=cross_grid.set_index("sid").loc[stock_pool].reset_index()[['sid','5d_count','10d_count']]
    cross_grid["date"]=date
    
    df_cross_grid_count=pd.concat([df_cross_grid_count,cross_grid])

df_cross_grid_count=df_cross_grid_count[(df_cross_grid_count["5d_count"]>=20) & (df_cross_grid_count["10d_count"]>=40)]
df_cross_grid_count=df_cross_grid_count.set_index("date").reset_index().sort_values(by=["date","sid"])

/home/ywang/env/dev/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  


In [37]:
def get_change_to_60d_high(sid,date):
    df_stk_price = get_stk_bar(sid,start=get_prev_n_trade_date(date,60),\
                               end = get_previous_trade_date(date),fields =["adj_high","adj_close","adj_factor"])
    price_60d_high =    np.nanmax(df_stk_price["adj_high"])
    adj_factor_yd = df_stk_price["adj_factor"].iloc[-1]
    price_yd_close = df_stk_price["adj_close"].iloc[-1]/ adj_factor_yd    
    
    with BarReader('/home/ywang/proj_cross_grid/reader/bar.yaml') as client:
        bars = client.stock_bars("000001")
        hq_df = pd.DataFrame(bars)
        pre_close = hq_df["preclose"].iloc[0]
        adj_factor_td = adj_factor_yd*price_yd_close/pre_close
        adj_high_td =  np.nanmax(hq_df["high"])*adj_factor_td
        adj_close_td = hq_df["close"].dropna().iloc[-1]*adj_factor_td

    change_to_60d_high = (max(price_60d_high,adj_high_td) - adj_close_td )/ max(price_60d_high,adj_high_td)
    
    return change_to_60d_high

cross_grid_summary_to_renew["change_to_60d_high"] = cross_grid_summary_to_renew[["sid","date"]]\
                                            .apply(lambda x:get_change_to_60d_high(x[0],x[1]),axis = 1)
cross_grid_summary_to_renew = cross_grid_summary_to_renew.sort_values("change_to_60d_high")

In [ ]:
%run stock_pool_renew_v240.py

In [ ]:
df= pd.read_excel(r"result/穿网格五天大于20十天大于40_白马_全部.xlsx",dtype={"date":str})
df= df.sort_values(["date","change_to_60d_high"])
df

In [50]:
df.to_excel(r"result/穿网格五天大于20十天大于40_白马_全部.xlsx",index=False)

## 记录第一次涨跌10%的日期（只记20d之内涨跌10%的情况，20天内连续触发只记第一次）

In [ ]:
#日期筛选 + 上下跌破标注
group = df_cross_grid_count.groupby(["sid","date"])
df_date_stop_pl=apply_parallel(group,get_date_stop_pl).reset_index().\
                rename(columns={'level_0':'sid',"level_1":"date"}).drop(['level_2'],axis=1)
df_cross_grid_count_1=pd.merge(df_cross_grid_count,df_date_stop_pl,on=["sid","date"])
df_cross_grid_count_1.to_excel(r"result/穿网格五天大于20十天大于40_白马_全部.xlsx",index=False)

#df_cross_grid_count=df_cross_grid_count.drop(['date_10p_ud','up_down'],axis=1)
df_cross_grid_count_2 = df_cross_grid_count.groupby("sid").apply(date_filter).reset_index(drop=True)
group = df_cross_grid_count_2.groupby(["sid","date"])
df_date_stop_pl=apply_parallel(group,get_date_stop_pl).reset_index().\
                rename(columns={'level_0':'sid',"level_1":"date"}).drop(['level_2'],axis=1)

df_cross_grid_count_2=pd.merge(df_cross_grid_count_2,df_date_stop_pl,on=["sid","date"])
df_cross_grid_count_2.to_excel(r"result/穿网格五天大于20十天大于40_白马_20天内不重复触发.xlsx",index=False)

## 改变止损止盈点，观察胜率变化

In [ ]:
df_win_ratio = pd.DataFrame(index=np.linspace(0.10,0.30,21),columns=["win_ratio","count"])

for ratio in np.linspace(0.10,0.30,21):
    #df_cross_grid_count=df_cross_grid_count.drop(['date_10p_ud','up_down'],axis=1)
    df_cross_grid_count_2 = df_cross_grid_count.groupby("sid").apply(date_filter).reset_index(drop=True)
    group = df_cross_grid_count_2.groupby(["sid","date"])
    df_date_stop_pl=apply_parallel(group,get_date_stop_pl,ratio=ratio).reset_index().\
                    rename(columns={'level_0':'sid',"level_1":"date"}).drop(['level_2'],axis=1)
    df_cross_grid_count_2=pd.merge(df_cross_grid_count_2,df_date_stop_pl,on=["sid","date"])
    df_win_ratio.loc[ratio,"win_ratio"]=len(df_cross_grid_count_2[df_cross_grid_count_2["up_down"]==1])\
                                /len(df_cross_grid_count_2[df_cross_grid_count_2["up_down"]==-1])
    df_win_ratio.loc[ratio,"count"] = len(df_cross_grid_count_2[pd.notnull(df_cross_grid_count_2["up_down"])])

In [ ]:
df_win_ratio["win_ratio"]=df_win_ratio["win_ratio"]/(1+df_win_ratio["win_ratio"])
df_win_ratio["total_profit"]= df_win_ratio["count"] * (2*df_win_ratio["win_ratio"]-1)

## 统计底仓股票日内穿网格的次数满足要求的日期

In [5]:
def get_stk_data(sid, date):
    
    try:
        raw_bar_dtype = [
            ('sid', '<U16'),
            ('DataDate', 'i4'),
            ('ticktime', 'i4'),
            ('pre_close', 'f8'),
            ('open', 'f8'),
            ('high', 'f8'),
            ('low', 'f8'),
            ('close', 'f8'),
            ('volume', 'f8'),
            ('amount', 'f8'),
            ('bid_amount', 'f8'),
            ('ask_amount', 'f8'),
            ('vwap', 'f8'),
            ('twap', 'f8'),
            ('ret', 'f8')
        ]


        bar=np.memmap(r"/share/intern_share/dat/stk_bar/1min/2019/{}/{}.dat".format(date,sid),dtype=raw_bar_dtype,mode="r")
        stock_data=pd.DataFrame(bar)[["sid","DataDate","ticktime","open", "high", "low", "close","pre_close"]]
        stock_data["ticktime"]=stock_data.index
        return stock_data
    except:
        try:
            stock_data = get_stk_bar(sid, freq="1m", start=date, end=date, fields=["open", "high", "low", "close"])
            stock_data = stock_data.reset_index().rename(columns={"index": "datetime"})
            stock_data["ticktime"] = stock_data["datetime"].index
            stock_data["DataDate"] = stock_data["datetime"].apply(lambda x:str(x.date()))
            del stock_data["datetime"]
            stock_data["pre_close"] = get_stk_bar(sid, freq="1d", start=date, end=date, fields=["pre_close"]).values[0][0]
            stock_data["sid"] = sid
            return stock_data
        except:
            pass


def get_daily_data(date):
    df=query_table("DailyBar",start_date=date,end_date=date,fields=["tradable"])
    stk_ls=list(df["sid"][df["tradable"]==1])

    results = Parallel(n_jobs=16, verbose=5, backend="loky", batch_size='auto') \
        (delayed(get_stk_data)(sid, date) for sid in stk_ls)
    data = pd.concat(results)

    return data.sort_values(["sid", "ticktime"])

def get_intraday_decre(df):
    price_low = np.min(df["low"].iloc[0:220])
    pre_close = df["pre_close"].iloc[0]
    return price_low/pre_close-1

## V1--2:40分界点

In [ ]:
trade_dates=get_trade_dates('20170301',"20191224")
trade_dates=[str(i) for i in trade_dates]

df_cross_grid_count_240=pd.DataFrame()
for date in trade_dates:
    stock_pool=pd.read_csv(r"/share/xfzhang/to_colleague/to_yzhao/task2_v2/{}/{}/task2_v2_{}.csv".format(date[0:4],date[4:6],date[0:8]))["sid"]
    
    df_daily_price=get_daily_data(date).set_index("sid").loc[stock_pool].reset_index()
    df_intraday_decre = df_daily_price.groupby("sid").apply(get_intraday_decre).to_frame("intraday_decre")
    stock_pool = df_intraday_decre[df_intraday_decre["intraday_decre"] < -0.03].reset_index()
    
    cross_grid=pd.read_excel(r'/home/ywang/proj_cross_grid/result/cross_count_b240/crosscount_summary_{}.xlsx'.format(date))
    cross_grid=cross_grid.merge(stock_pool,on='sid')[['sid','5d_count','10d_count',"intraday_decre"]]
    cross_grid["date"]=date
    
    df_cross_grid_count_240=pd.concat([df_cross_grid_count_240,cross_grid])

df_cross_grid_count_240=df_cross_grid_count_240[(df_cross_grid_count_240["5d_count"]>=50) & (df_cross_grid_count_240["10d_count"]>=100)]
df_cross_grid_count_240=df_cross_grid_count_240.set_index("date").reset_index().sort_values(by=["sid","date"])

## V2--3:00分界点

In [6]:

trade_dates=get_trade_dates('20170222',"20191229")
trade_dates=[str(i) for i in trade_dates]

df_cross_grid_count_300=pd.DataFrame()
for date in trade_dates:
    stock_pool=pd.read_csv(r"/share/xfzhang/to_colleague/to_yzhao/task2_v2/{}/{}/task2_v2_{}.csv".format(date[0:4],date[4:6],date[0:8]))["sid"]
    
    df_daily_price=query_table("DailyBar",date,date,fields=["pre_close","low"])
    df_daily_price=df_daily_price.set_index("sid").loc[stock_pool.values].reset_index()
    df_daily_price["intraday_decre"] = df_daily_price["low"] / df_daily_price["pre_close"]-1
    stock_pool = df_daily_price[["sid","intraday_decre"]][df_daily_price["intraday_decre"]<-0.03]
    
    cross_grid=pd.read_excel(r'/share/intern_share/stk_bundle_v2/crosscount_summary_{}.xlsx'.format(date))
    cross_grid=cross_grid.merge(stock_pool,on='sid')[['sid','5d_count','10d_count',"intraday_decre"]]
    cross_grid["date"]=date
    
    df_cross_grid_count_300=pd.concat([df_cross_grid_count_300,cross_grid])

df_cross_grid_count_300=df_cross_grid_count_300[(df_cross_grid_count_300["5d_count"]>=50) & (df_cross_grid_count_300["10d_count"]>=100)]
df_cross_grid_count_300=df_cross_grid_count_300.set_index("date").reset_index().sort_values(by=["sid","date"])

/home/ywang/env/dev/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  if __name__ == '__main__':


## 当日或者前一日涨跌停股票

In [ ]:

df_cross_count = pd.read_excel(r"result/穿网格五天大于50十天大于100盘中跌超3%_底仓_nodatefilter_17_19_止盈止损15%.xlsx")
df_if_limit= query_table("DailyBar",start_date="20170101",end_date="20191224",fields=["if_limit"])
df_if_limit_today = df_if_limit.rename(columns={"DataDate":"date","if_limit":"if_limit_today"})
df_cross_count = df_cross_count.merge(df_if_limit_today,on=["sid","date"])
"""df_if_limit = df_if_limit.rename(columns={"DataDate":"date_{}p_ud".format(percent)})
df_cross_count = df_cross_count.merge(df_if_limit,on=["sid","date_{}p_ud".format(percent)])
df_cross_count = df_cross_count[df_cross_count["if_limit"]==0.0]"""
def get_next_date(df):
    df=df.rename(columns={"DataDate":"date"})
    date=int(get_next_trade_date(df["date"].iloc[0]))
    df["date"] = date
    return df.rename(columns={"if_limit":"if_limit_yesterday"})
df_if_limit_yesterday = df_if_limit.groupby("DataDate").apply(get_next_date)
df_cross_count = df_cross_count.merge(df_if_limit_yesterday,on=["sid","date"])

df_cross_count_drop = df_cross_count[(df_cross_count["if_limit_today"]!=0.0) | (df_cross_count["if_limit_yesterday"]!=0.0)]
df_cross_count_drop.to_excel(r"result/穿网格五天大于50十天大于100盘中跌超3%_底仓_止盈止损15%_当日或前一日涨跌停股票.xlsx")

## 记录第一次涨跌10%的日期（只记20d之内涨跌10%的情况，20天内连续触发只记第一次）

In [ ]:
## 日期筛选 + 上下跌破标注
ratio=0.10
group = df_cross_grid_count_300.groupby(["sid","date"])
df_date_stop_pl=apply_parallel(group,get_date_stop_pl,ratio=ratio).reset_index().\
                rename(columns={'level_0':'sid',"level_1":"date"}).drop(['level_2'],axis=1)
df_cross_grid_count_1=pd.merge(df_cross_grid_count_300,df_date_stop_pl,on=["sid","date"])
df_cross_grid_count_1.to_excel(r"result/穿网格五天大于50十天大于100盘中跌超3%_底仓_全部.xlsx",index=False)

#df_cross_grid_count_300=df_cross_grid_count_300.drop(['date_10p_ud','up_down'],axis=1)
df_cross_grid_count_2 = df_cross_grid_count_300.groupby("sid").apply(date_filter).reset_index(drop=True)
group = df_cross_grid_count_2.groupby(["sid","date"])
df_date_stop_pl=apply_parallel(group,get_date_stop_pl,ratio=ratio).reset_index().\
                rename(columns={'level_0':'sid',"level_1":"date"}).drop(['level_2'],axis=1)

df_cross_grid_count_2=pd.merge(df_cross_grid_count_2,df_date_stop_pl,on=["sid","date"])
df_cross_grid_count_2.to_excel(r"result/穿网格五天大于50十天大于100盘中跌超3%_底仓_20天内不重复触发.xlsx",index=False)

In [ ]:
%run stock_pool_renew.py

## 改变止损止盈点，观察胜率变化

In [ ]:
df_win_ratio = pd.DataFrame(index=np.linspace(0.10,0.20,3),columns=["win_ratio","count"])

for ratio in np.linspace(0.10,0.20,3):
    #df_cross_grid_count=df_cross_grid_count.drop(['date_10p_ud','up_down'],axis=1)
    #df_cross_grid_count_2 = df_cross_grid_count_240.groupby("sid").apply(date_filter).reset_index(drop=True)
    df_cross_grid_count_2 = df_cross_grid_count_300.copy()
    group = df_cross_grid_count_2.groupby(["sid","date"])
    df_date_stop_pl=apply_parallel(group,get_date_stop_pl,ratio=ratio).reset_index().\
                    rename(columns={'level_0':'sid',"level_1":"date"}).drop(['level_2'],axis=1)
    df_cross_grid_count_2=pd.merge(df_cross_grid_count_2,df_date_stop_pl,on=["sid","date"])
    df_win_ratio.loc[ratio,"win_ratio"]=len(df_cross_grid_count_2[df_cross_grid_count_2["up_down"]==1])\
                                /len(df_cross_grid_count_2[df_cross_grid_count_2["up_down"]==-1])
    df_win_ratio.loc[ratio,"count"] = len(df_cross_grid_count_2[pd.notnull(df_cross_grid_count_2["up_down"])])
    
    df_cross_grid_count_2.to_excel(r"result/穿网格五天大于50十天大于100盘中跌超3%_底仓_nodatefilter_17_19_止盈止损{}%.xlsx".format(int(ratio*100)),index=False)

In [ ]:
df_win_ratio["win_ratio"]=df_win_ratio["win_ratio"]/(1+df_win_ratio["win_ratio"])
df_win_ratio["total_profit"]= df_win_ratio["count"] * (2*df_win_ratio["win_ratio"]-1)
df_win_ratio

## 将白马股单独列出一个文件

In [ ]:
trade_dates=get_trade_dates('20190102',"20191230")

for date in trade_dates:
    stock_pool=pd.read_csv(r"/share/xfzhang/to_colleague/to_yzhao/task2/{}/{}/task2_{}.csv".format(date[0:4],date[4:6],date[0:8]))["sid"]
    cross_grid=pd.read_excel(r'/share/intern_share/stk_bundle_v2/crosscount_summary_{}.xlsx'.format(date))
    cross_grid = cross_grid.set_index("sid").loc[stock_pool]
    
    cross_grid.to_excel(r'/share/intern_share/stk_bundle_baima/crosscount_summary_{}.xlsx'.format(date))

## 白马池回测，5d、10d同时高于买入，同时低于卖出

In [21]:
from data_tools.api import trade_days
trade_dates_all = trade_days.copy()
def get_prev_n_trade_date(trade_date, n):
    pos = np.searchsorted(trade_dates_all, trade_date)
    assert pos >= n
    return int(trade_dates_all[pos - n])

def get_next_n_trade_date(trade_date, n=1):
    pos = np.searchsorted(trade_dates_all, trade_date, side='right')
    assert pos + n - 1 < len(trade_dates_all)
    return int(trade_dates_all[pos + n - 1])

In [ ]:
trade_dates=get_trade_dates('20170222',"20191230")
trade_dates=[str(i) for i in trade_dates]

df_cross_grid_count=pd.DataFrame()
for date in trade_dates:
    stock_pool=pd.read_csv(r"/share/xfzhang/to_colleague/to_yzhao/task2/{}/{}/task2_{}.csv".format(date[0:4],date[4:6],date[0:8]))["sid"]
    cross_grid=pd.read_excel(r'/share/intern_share/stk_bundle_v2/crosscount_summary_{}.xlsx'.format(date))
    cross_grid=cross_grid.set_index("sid").loc[stock_pool].reset_index()[['sid','5d_count','10d_count']]
    cross_grid["date"]=date
    
    df_cross_grid_count=pd.concat([df_cross_grid_count,cross_grid])

df_stk_pool_buy=df_cross_grid_count[(df_cross_grid_count["5d_count"]>=20) & (df_cross_grid_count["10d_count"]>=40)]
df_stk_pool_buy=df_stk_pool_buy.set_index("date").reset_index().sort_values(by=["sid","date"])

df_stk_pool_sell=df_cross_grid_count[(df_cross_grid_count["5d_count"]<20) & (df_cross_grid_count["10d_count"]<40)]
df_stk_pool_sell=df_stk_pool_sell.set_index("date").reset_index().sort_values(by=["sid","date"])

In [ ]:
stk_ls_holding = []
port_ret_daily = pd.Series(index=pd.to_datetime(trade_dates[1:]))
port_stk_num = pd.Series(index=pd.to_datetime(trade_dates[1:]))

def get_daily_ret(stk_ls_holding,date,trade_calendar):
    next_date= get_next_trade_date(date)
    
    today_price=query_table("DailyBar",start_date=date,end_date=date,fields=["tradable","adj_close"])
    today_price=today_price[today_price["tradable"]==1]
    
    next_date_price=query_table("DailyBar",start_date=next_date,end_date=next_date,fields=["tradable","adj_close"])
    #next_date_price=next_date_price[next_date_price["tradable"]==1]
    
    df_p_10d_price = query_table("DailyBar",start_date=get_prev_n_trade_date(date,10),end_date=get_prev_n_trade_date(date,10),fields=["tradable","adj_close"])
    #df_p_10d_price = df_p_10d_price[df_p_10d_price["tradable"]==1]
    
    merged_price = pd.merge(df_p_10d_price,today_price,on= "sid" )
    stk_incre = merged_price[merged_price["adj_close_x"]<merged_price["adj_close_y"]].sid
    
    stk_ls_holding = list(set(stk_ls_holding) & set(stk_incre))
    
    merged_price=pd.merge(today_price,next_date_price,on="sid")
    merged_price["change_rate"]=merged_price["adj_close_y"]/merged_price["adj_close_x"]-1
    
    merged_price = merged_price.set_index("sid").loc[stk_ls_holding].reset_index()
    
    
    return  np.nanmean(merged_price["change_rate"]),len(stk_ls_holding)

for date in trade_dates[:-1]:
    print(date)
    stk_to_buy= df_stk_pool_buy[df_stk_pool_buy["date"] == date].sid.values
    stk_to_sell= df_stk_pool_sell[df_stk_pool_sell["date"] == date].sid.values
    
    stk_ls_holding = list(set(stk_ls_holding) - set(stk_to_sell))
    stk_ls_holding = list(set(stk_ls_holding) | set(stk_to_buy))
    
    port_ret_daily.loc[get_next_trade_date(date)], \
    port_stk_num.loc[get_next_trade_date(date)] = \
    get_daily_ret(stk_ls_holding,date,trade_dates)
   
    

In [ ]:
plt.plot(np.cumprod(port_ret_daily+1),label = "cross_grid" )
plt.xticks(rotation=90)

sql="select S_INFO_WINDCODE,TRADE_DT,S_DQ_CLOSE FROM AINDEXEODPRICES WHERE S_INFO_WINDCODE ='000300.SH'"
index_hq=pd.read_sql(sql,conn)
index_hq.sort_values("TRADE_DT",inplace=True)
index_hq = index_hq.set_index("TRADE_DT").loc["20170222":"20191230"].reset_index()
plt.plot(pd.to_datetime(index_hq["TRADE_DT"]),index_hq["S_DQ_CLOSE"]/index_hq["S_DQ_CLOSE"][index_hq.index[0]],label="hs300")

sql="select S_INFO_WINDCODE,TRADE_DT,S_DQ_CLOSE FROM AINDEXEODPRICES WHERE S_INFO_WINDCODE ='000905.SH'"
index_hq=pd.read_sql(sql,conn)
index_hq.sort_values("TRADE_DT",inplace=True)
index_hq = index_hq.set_index("TRADE_DT").loc["20170222":"20191230"].reset_index()
plt.plot(pd.to_datetime(index_hq["TRADE_DT"]),index_hq["S_DQ_CLOSE"]/index_hq["S_DQ_CLOSE"][index_hq.index[0]],label="cs500")

plt.legend()
plt.grid()
plt.show()

In [ ]:
def get_max_drawdown(arr):
    max_drawdown=0
    for i in range(len(arr)):
        cum_ret=np.cumprod(1+arr[i:])
        if (1 - np.min(cum_ret)) > max_drawdown:
            max_drawdown = 1 - np.min(cum_ret)
    return max_drawdown

def get_indicator_df(sid,df):    
    indicator_df=pd.DataFrame(index=pd.MultiIndex.from_product([sid,["stock","strategy","strategy_adj"]]),\
                          columns=["tot_return","yearly_ret","daily_volat","yearly_volat","sharpe","yearly_sharpe","max_drawdown"])
    indicator_df.loc[(sid,"stock"),"tot_return"] = df["stock_ret"].iloc[-1]-1
    indicator_df.loc[(sid,"strategy"),"tot_return"] = df["strategy_ret"].iloc[-1]-1
    indicator_df.loc[(sid,"stock"),"yearly_ret"] = np.power(df["stock_ret"].iloc[-1],252/len(df))-1
    indicator_df.loc[(sid,"strategy"),"yearly_ret"] = np.power(df["strategy_ret"].iloc[-1],252/len(df))-1
    indicator_df.loc[(sid,"stock"),"daily_volat"] = np.nanstd(df["change_rate"])
    indicator_df.loc[(sid,"strategy"),"daily_volat"] = np.nanstd(df["trading_profit_loss"])
    indicator_df.loc[(sid,"stock"),"yearly_volat"] = np.nanstd(df["change_rate"])*np.sqrt(252)
    indicator_df.loc[(sid,"strategy"),"yearly_volat"] = np.nanstd(df["trading_profit_loss"])*np.sqrt(252)
    indicator_df.loc[(sid,"stock"),"sharpe"] = np.nanmean(df["stock_ret"]) / np.nanstd(df["stock_ret"])
    indicator_df.loc[(sid,"strategy"),"sharpe"] = np.nanmean(df["strategy_ret"]) / np.nanstd(df["strategy_ret"])
    indicator_df.loc[(sid,"stock"),"yearly_sharpe"] = indicator_df.loc[(sid,"stock"),"yearly_ret"] / indicator_df.loc[(sid,"stock"),"yearly_volat"]
    indicator_df.loc[(sid,"strategy"),"yearly_sharpe"] = indicator_df.loc[(sid,"strategy"),"yearly_ret"] / indicator_df.loc[(sid,"strategy"),"yearly_volat"]
    indicator_df.loc[(sid,"stock"),"max_drawdown"] = get_max_drawdown(df["change_rate"].dropna().values)
    indicator_df.loc[(sid,"strategy"),"max_drawdown"] =  get_max_drawdown(df["trading_profit_loss"].dropna().values) 
    return indicator_df

In [ ]:
plt.plot(np.cumprod(port_ret_daily+1),label = "cross_grid" )
plt.xticks(rotation=90)

sql="select S_INFO_WINDCODE,TRADE_DT,S_DQ_CLOSE FROM AINDEXEODPRICES WHERE S_INFO_WINDCODE ='000300.SH'"
index_hq=pd.read_sql(sql,conn)
index_hq.sort_values("TRADE_DT",inplace=True)
index_hq = index_hq.set_index("TRADE_DT").loc["20170222":"20191230"].reset_index()
plt.plot(pd.to_datetime(index_hq["TRADE_DT"]),index_hq["S_DQ_CLOSE"]/index_hq["S_DQ_CLOSE"][index_hq.index[0]],label="hs300")

plt.legend()
plt.grid()
plt.show()

In [ ]:
index_hq

In [ ]:
get_indicator_df(["portfolio"],overall_ret)